In [1]:
## creating a venv
# !pip install virtualenv
# !virtualenv statconsvenv

In [2]:
## initiate venv
# !source statconsvenv/bin/activate

In [3]:
# !pip install -r requirements.txt
# !pip freeze > requirements.txt

In [4]:
# !pip install circlify
# !pip install folium
# !pip install squarify
# !pip install mpl_extra
# !pip install calplot
# !pip install plotly-calplot

In [5]:
# !pip list

In [6]:
# import packages
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import datetime
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from platform import python_version
from pprint import pprint as pp
import sys
import folium    
from collections import Counter
import plotly
# from plotly_calplot import calplot
import circlify as circ
import squarify
from matplotlib.pyplot import figure
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn import preprocessing

In [7]:
path = 'https://www.dropbox.com/s/9p11vj3svblow6j/df_final.csv?dl=1'

In [8]:
path2 = 'https://www.dropbox.com/s/pvc1azze3p0yq6p/abbr-name.csv?dl=1'

In [ ]:
df_final = pd.read_csv(path, header=0)

In [ ]:
df_statenames = pd.read_csv(path2, header=0)

In [ ]:
list(df_final)

In [ ]:
or_state = []
full_or_state = []

In [ ]:
for i in range(len(df_statenames)):
    x = df_statenames[';OR_STATE;FULL_STATE'][i].split(";")
    or_state.append(x[1])
    full_or_state.append(x[2])

In [ ]:
df_state = pd.DataFrame()

In [ ]:
df_state['OR_STATE'] = or_state
df_state['OR_FULL_STATE'] = full_or_state

In [ ]:
list(df_final)

In [ ]:
# creating a df with only unique values to do visualization of the flight paths
unique = pd.DataFrame()

unique['OR_LAT'] = df_final['OR_LATITUDE'].unique()
unique['OR_LONG'] = df_final['OR_LONGITUDE'].unique()
unique['DES_LAT'] = df_final['DES_LATITUDE'].unique()
unique['DES_LONG'] = df_final['DES_LONGITUDE'].unique()
unique['OR_AIRPORT'] = df_final['OR_AIRPORT_NAME'].unique()
unique['DES_AIRPORT'] = df_final['DES_AIRPORT_NAME'].unique()

In [ ]:
# combining the OR/DES coordinates for plot use
unique["OR_COORDS"] = list(zip(unique['OR_LAT'], unique['OR_LONG']))
unique["DES_COORDS"] = list(zip(unique['DES_LAT'], unique['DES_LONG']))
unique["FLIGHT_PATHS"] = list(zip(unique["OR_COORDS"], unique["DES_COORDS"]))

In [ ]:
# all unique flight paths
flight_paths = list(unique['FLIGHT_PATHS'])

In [ ]:
# using only unique flights so you can use the number of same flights 
# as a variable or can average out delays for same path flight\

combo_unique = pd.DataFrame(flight_paths)

In [ ]:
# renaming columns
combo_unique.rename(columns={combo_unique.columns[0]: "OR_AIRPORT", combo_unique.columns[1]: "DES_AIRPORT"}, inplace = True)
# combo_unique.head()

In [ ]:
# len(combo_unique)

In [ ]:
# creating a list of all coordinate pairs
all_org_coords = list(zip(df_final['OR_LATITUDE'], df_final['OR_LONGITUDE']))
all_des_coords = list(zip(df_final['DES_LATITUDE'], df_final['DES_LONGITUDE']))

In [ ]:
# combining all coordinate pairs
final_coords = list(zip(all_org_coords, all_des_coords))

In [ ]:
coords_final = pd.DataFrame(final_coords)

In [ ]:
# all flights in dataset
coords_final.rename(columns={coords_final.columns[0]: "OR_AIRPORT", coords_final.columns[1]: "DES_AIRPORT"}, inplace = True)
coords_final.head()

In [ ]:
df_final = pd.concat([df_final, coords_final], axis=1)
df_final.head()

In [ ]:
flight_count_by_airline = df_final.sort_values(by = ['OR_AIRPORT'], ascending = False)
flight_count_by_airline = flight_count_by_airline.reset_index(drop = True)

In [ ]:
flight_count_by_airline = flight_count_by_airline.groupby(['ORIGIN_AIRPORT','AIRLINE'])["FLIGHT_NUMBER"].count()


In [ ]:
flight_count_by_airline = flight_count_by_airline.to_frame()

In [ ]:
# combining coordinates for the similar flight paths
df_flight_grouped = df_final.groupby(['OR_AIRPORT','DES_AIRPORT'], as_index=False)['ARRIVAL_DELAY_v1'].mean()
# df_flight_grouped.head()

In [ ]:
df_flight_grouped['ARRIVAL_DELAY'] = list(df_flight_grouped['ARRIVAL_DELAY_v1'].round(2))

In [ ]:
# max delay
df_flight_grouped['ARRIVAL_DELAY_v1'].max()

In [ ]:
# min delay
df_flight_grouped['ARRIVAL_DELAY_v1'].min()

In [ ]:
# df_flight_grouped.head()

In [ ]:
# len(df_flight_grouped)

In [ ]:
# identify delay and total number of flights by airline

unique_airlines = df_final['AIRLINE'].unique()
df_airline_group = pd.DataFrame(unique_airlines)

In [ ]:
df_airline_group = df_airline_group.sort_values([0])

In [ ]:
mean_ad = df_final.groupby(['AIRLINE'])['ARRIVAL_DELAY_v1'].mean()
mean_d = list(mean_ad)
df_airline_group['MEAN_DEP_DELAY'] = mean_d
# df_airline_group.head()

In [ ]:
flights_by_airline = df_final['AIRLINE'].value_counts()
flights_by_airline = flights_by_airline.sort_index()
air_flight = list(flights_by_airline)
df_airline_group['NUMBER_OF_FLIGHTS'] = air_flight
# df_airline_group

In [ ]:
df_airline_group.rename(columns={df_airline_group.columns[0]: 'AIRLINE'}, inplace=True)

In [ ]:
# average delays by airline and how many flights they've flown in the dataset
df_airline_group.sort_values('NUMBER_OF_FLIGHTS', ascending=False)

In [ ]:
# identify delay and total number of flights by airport
df_airport_group = df_final.groupby(['ORIGIN_AIRPORT', 'OR_STATE'])['ARRIVAL_DELAY_v1'].mean()
df_airport_group = df_airport_group.to_frame().reset_index()


In [ ]:
flights_by_airport = df_final['ORIGIN_AIRPORT'].value_counts()
flights_by_airport = flights_by_airport.sort_index()
flights_by_airport = flights_by_airport.tolist()

In [ ]:
df_airport_group['NUM_OF_FLIGHTS'] = flights_by_airport

In [ ]:
df_airport_group.head()

In [ ]:
# len(df_airport_group)

In [ ]:
# airports and avg delay sorted by busiest/most flights)
# could visualize this as size and color circles (size = num flights, color = avg delay time)

df_airport_group.sort_values(by=['NUM_OF_FLIGHTS'], ascending=False)

In [ ]:
# data frame of airport information

df_airport_group = df_airport_group.reset_index(drop=True)
df_airport_group = df_airport_group.rename(columns={'ORIGIN_AIRPORT': 'AIRPORT'})

In [ ]:
# merging with the full name rather than the abbreviation
df_airport_group = pd.merge(df_airport_group, df_state, on='OR_STATE', how='outer')

In [ ]:
df_airport_group.head()

In [ ]:
# count of flights by airlines in airports

df_airport_airline = pd.DataFrame()
df_alt_air = pd.DataFrame()

In [ ]:
df_airport_airline['AIRLINE'] = df_final['AIRLINE']
df_airport_airline['ORIGIN_AIRPORT'] = df_final['ORIGIN_AIRPORT']
df_airport_airline['ARRIVAL_DELAY'] = df_final['ARRIVAL_DELAY_v1']
# df_airport_airline["TAIL_NUMBER"] = df_final["TAIL_NUMBER"]

In [ ]:
df_alt_air['AIRLINE'] = df_final['AIRLINE']
df_alt_air['ORIGIN_AIRPORT'] = df_final['ORIGIN_AIRPORT']
df_alt_air["TAIL_NUMBER"] = df_final["TAIL_NUMBER"]

In [ ]:
df_airport_airline = df_airport_airline.groupby(["ORIGIN_AIRPORT", "AIRLINE"])["ARRIVAL_DELAY"].mean()

In [ ]:
airline_flight_count = df_alt_air.groupby(["ORIGIN_AIRPORT", "AIRLINE"])["TAIL_NUMBER"].count()

In [ ]:
airline_flight_count = pd.DataFrame(airline_flight_count)

In [ ]:
airline_flight_count.head()

In [ ]:
df_airport_airline = pd.DataFrame(df_airport_airline)

In [ ]:
df_airport_airline.head()

In [ ]:
# trying to organize data into a dictionary to use circlify
circledata = df_airport_airline.to_dict('dict')
circledata = circledata['ARRIVAL_DELAY']

In [ ]:
keys = circledata.keys()
values = circledata.values()

In [ ]:
airport_airline = []
for entry in keys:
    case = {'AIRPORT': entry[0], 'AIRLINE': entry[1]}
    airport_airline.append(case)

In [ ]:
# range(len(airport_airline))

In [ ]:
# airport_airline[0]

In [ ]:
delays = list(values)

In [ ]:
# delays

In [ ]:
for i in range(len(airport_airline)):
    airport_airline[i]['AVG_DELAY'] = delays[i]

In [ ]:
airports = []
airlines = []
delay_times = []

In [ ]:
for y in range(len(airport_airline)):
    airports.append(airport_airline[y]['AIRPORT'])
    airlines.append(airport_airline[y]['AIRLINE'])
    delay_times.append(airport_airline[y]['AVG_DELAY'])

In [ ]:
# delay_times

In [ ]:
airport_name = Counter(airports).keys()
airline_count = Counter(airports).values()

In [ ]:
dict_df = pd.DataFrame.from_dict(airport_airline)

In [ ]:
# this is basically the same thing as df_airport_airline just not grouped
# dict_df

In [ ]:
avg_delays = df_airport_group['ARRIVAL_DELAY_v1']

In [ ]:
df_airport_delays = pd.merge(df_airport_group, dict_df, on='AIRPORT', how='right')
df_airport_delays = df_airport_delays.rename(columns={'ARRIVAL_DELAY_v1': 'AVG_DELAY_BY_AIRPORT', 'AVG_DELAY': 'AVG_DELAY_BY_AIRLINE'})

In [ ]:
df_airport_delays = df_airport_delays.round({'AVG_DELAY_BY_AIRPORT': 2, 'AVG_DELAY_BY_AIRLINE': 2})
df_airport_delays = df_airport_delays.rename(columns={'NUM_OF_FLIGHTS': 'NUM_OF_FLIGHTS_BY_AIRPORT'})

In [ ]:
df_airport_delays['NUM_OF_FLIGHTS_BY_AIRLINE_AIRPORT'] = list(airline_flight_count["TAIL_NUMBER"])

In [ ]:
df_airport_delays['DELAY_RATIO'] = df_airport_delays['AVG_DELAY_BY_AIRLINE']/df_airport_delays['AVG_DELAY_BY_AIRPORT']

In [ ]:
# df_airport_delays['DELAY_RATIO'] = df_airport_delays.loc[df_airport_delays['DELAY_RATIO'] <= -1].replace(-1)
df_airport_delays.loc[df_airport_delays['DELAY_RATIO'] < -15, 'DELAY_RATIO'] = -15
df_airport_delays.loc[df_airport_delays['DELAY_RATIO'] >= 15, 'DELAY_RATIO'] = 15

In [ ]:
df_delay_ratio = df_airport_delays.groupby(["AIRPORT"])["DELAY_RATIO"].sum()

In [ ]:
df_delay_ratio = pd.DataFrame(df_delay_ratio)
df_delay_ratio.rename(columns={'DELAY_RATIO': 'DELAY_RATIO_SUM_AIRPORT'}, inplace=True)

In [ ]:
df_airport_delays = pd.merge(df_airport_delays, df_delay_ratio, on='AIRPORT', how='left')

In [ ]:
df_airport_delays['DUMMY_AIRPORT_DELAY'] = df_airport_delays['AVG_DELAY_BY_AIRPORT']
df_airport_delays['DUMMY_AIRLINE_DELAY'] = df_airport_delays['AVG_DELAY_BY_AIRLINE']

In [ ]:
df_airport_delays.loc[df_airport_delays['DUMMY_AIRPORT_DELAY'] <= -15 , 'DUMMY_AIRPORT_DELAY'] = -1
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRPORT_DELAY'] > -15) & (df_airport_delays['DUMMY_AIRPORT_DELAY'] < 0) , 'DUMMY_AIRPORT_DELAY'] = -0.5
df_airport_delays.loc[df_airport_delays['DUMMY_AIRPORT_DELAY'] == 0 , 'DUMMY_AIRPORT_DELAY'] = 0
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRPORT_DELAY'] > 0) & (df_airport_delays['DUMMY_AIRPORT_DELAY'] <= 15), 'DUMMY_AIRPORT_DELAY'] = 0.5
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRPORT_DELAY'] > 15) & (df_airport_delays['DUMMY_AIRPORT_DELAY'] <= 30) , 'DUMMY_AIRPORT_DELAY'] = 1
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRPORT_DELAY'] > 30) & (df_airport_delays['DUMMY_AIRPORT_DELAY'] <= 45) , 'DUMMY_AIRPORT_DELAY'] = 1.5
df_airport_delays.loc[df_airport_delays['DUMMY_AIRPORT_DELAY'] > 45 , 'DUMMY_AIRPORT_DELAY'] = 2

In [ ]:
df_airport_delays.loc[df_airport_delays['DUMMY_AIRLINE_DELAY'] <= -15 , 'DUMMY_AIRLINE_DELAY'] = -1
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRLINE_DELAY'] > -15) & (df_airport_delays['DUMMY_AIRLINE_DELAY'] < 0) , 'DUMMY_AIRLINE_DELAY'] = -0.5
df_airport_delays.loc[df_airport_delays['DUMMY_AIRLINE_DELAY'] == 0 , 'DUMMY_AIRLINE_DELAY'] = 0
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRLINE_DELAY'] > 0) & (df_airport_delays['DUMMY_AIRLINE_DELAY'] <= 15), 'DUMMY_AIRLINE_DELAY'] = 0.5
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRLINE_DELAY'] > 15) & (df_airport_delays['DUMMY_AIRLINE_DELAY'] <= 30) , 'DUMMY_AIRLINE_DELAY'] = 1
df_airport_delays.loc[(df_airport_delays['DUMMY_AIRLINE_DELAY'] > 30) & (df_airport_delays['DUMMY_AIRLINE_DELAY'] <= 45) , 'DUMMY_AIRLINE_DELAY'] = 1.5
df_airport_delays.loc[df_airport_delays['DUMMY_AIRLINE_DELAY'] > 45 , 'DUMMY_AIRLINE_DELAY'] = 2

In [ ]:
df_airport_delays.head()

Visualizations

In [ ]:
# make a map of all the flight paths

US_flight_map = folium.Map(location=[unique['OR_LAT'].mean() + 12, unique['OR_LONG'].mean()], 
                 zoom_start=3, control_scale=True)

In [ ]:
# placing the markers of the airports
for index, org_airports in unique.iterrows():
    folium.Circle([org_airports["DES_LAT"], org_airports["DES_LONG"]], popup=org_airports["DES_AIRPORT"], radius=4, color='skyblue').add_to(US_flight_map)

In [ ]:
points = []
for i in range(len(df_flight_grouped)):
    points.append([df_flight_grouped['OR_AIRPORT'][i], df_flight_grouped['DES_AIRPORT'][i]])


In [ ]:
# mapping the flight paths
folium.PolyLine(points,
                color='forestgreen',
                weight=1,
                opacity=0.1).add_to(US_flight_map)

In [ ]:
# US_flight_map

In [ ]:
# df_airport_delays.head()

In [ ]:
# tree map for airports delays taking in account size differences
fig = px.treemap(df_airport_delays, 
                 path=[px.Constant("US AIRPORTS"), 'OR_STATE', 'AIRPORT'], 
                 values='NUM_OF_FLIGHTS_BY_AIRPORT',
                 color='AVG_DELAY_BY_AIRPORT',
                 color_continuous_scale='balance',
                 color_continuous_midpoint=np.average(df_airport_delays['AVG_DELAY_BY_AIRPORT'], weights=df_airport_delays['NUM_OF_FLIGHTS_BY_AIRPORT']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()

In [ ]:
# tree map for airlines delay taking in account airport sizes

fig = px.treemap(df_airport_delays, 
                 path=[px.Constant("AIRPORTS IN THE US BY STATE THEN AIRPORT"), 'OR_STATE', 'AIRPORT', 'AIRLINE'], 
                 values='NUM_OF_FLIGHTS_BY_AIRLINE_AIRPORT',
                 color='DUMMY_AIRLINE_DELAY',
                 color_continuous_scale='balance',
                 color_continuous_midpoint=np.average(df_airport_delays['DUMMY_AIRLINE_DELAY'],))

fig.update_layout(coloraxis_colorbar=dict(title='Arrival Delay Times',tickvals = [2,1.5,1,.5,0,-.5,-1], ticktext = ['45+ min delay', '30-45 min delay', '15-30 min delay', '1-15 min delay', '0 min (on time)', '1-15 mins early', '15+ mins early']))
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show() 


In [ ]:
# chloropleth map
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"



m = folium.Map(location=[48, -102], zoom_start=3)

folium.Choropleth(
    geo_data = state_geo,
    name ="choropleth",
    data = df_airport_delays,
    columns = ["OR_STATE", "AVG_DELAY_BY_AIRPORT"],
    key_on = "feature.id",
    fill_color = "Reds",
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Average Delay Time",
).add_to(m)

folium.LayerControl().add_to(m)

# placing the markers of the airports
for index, org_airports in unique.iterrows():
    folium.Circle([org_airports["DES_LAT"], org_airports["DES_LONG"]], popup=org_airports["DES_AIRPORT"], radius=4, color='grey').add_to(m)
    

m

In [ ]:
airline_names = list(df_airport_delays['AIRLINE'])
airport_names = list(df_airport_delays['AIRPORT'])
airline_delay = list(df_airport_delays['AVG_DELAY_BY_AIRLINE'])

In [ ]:
df_airport_delays.head()

In [ ]:
fig = px.bar(df_airport_delays, x="AIRPORT", y='NUM_OF_FLIGHTS_BY_AIRPORT', color="AVG_DELAY_BY_AIRPORT", title="Long-Form Input")
# fig.show()

In [ ]:
fig = px.bar(df_airport_delays, x='AIRPORT', y='NUM_OF_FLIGHTS_BY_AIRLINE_AIRPORT',
             hover_data=['AVG_DELAY_BY_AIRLINE', 'AVG_DELAY_BY_AIRPORT'], color='AIRLINE',
             labels={'NUM_OF_FLIGHTS_BY_AIRLINE_AIRPORT':'Number of Flights in Airport'}, height=400)
fig.show()

In [ ]:
list(df_final)

In [ ]:
busiest_airports = pd.DataFrame(df_final)

In [ ]:
busiest_airports = busiest_airports.sort_values(by=['OR_AIRPORT'], ascending=False)
busiest_airports = busiest_airports.reset_index(drop=True)
busiest_airports.head()

In [ ]:
busiest_airports_count = busiest_airports.groupby("ORIGIN_AIRPORT")["TAIL_NUMBER"].count()

In [ ]:
busiest_airports_count = pd.DataFrame(busiest_airports_count)
calendar_delays = busiest_airports[['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT' ,'MONTH_DE', 'DAY_DE', 'WEEKEND_DE', 'HOUR_DE', 'TIME_AR','TIME_DE','ARRIVAL_DELAY_v1']].copy()

In [ ]:
busiest_airports_count.head()

In [ ]:
calendar_ar_delays = calendar_delays.groupby(["MONTH_DE", "DAY_DE"], as_index=False).mean()
calendar_ar_delays.head()

In [ ]:
count_calendar = calendar_delays.groupby(["MONTH_DE", "DAY_DE"]).size().reset_index(name="DATE_FLIGHT_COUNT")
count_calendar.head()

In [ ]:
count_calendar['DATE_AVG_AR_DELAY'] = calendar_delays['ARRIVAL_DELAY_v1']

In [ ]:
count_calendar.head()

In [ ]:
count_calendar['DATE_AVG_AR_DELAY'].min()

In [ ]:
count_calendar.loc[count_calendar['DATE_AVG_AR_DELAY'] <= -15 , 'DUMMY_DELAY'] = -1
count_calendar.loc[(count_calendar['DATE_AVG_AR_DELAY'] > -15) & (count_calendar['DATE_AVG_AR_DELAY'] < 0) , 'DUMMY_DELAY'] = -0.5
count_calendar.loc[count_calendar['DATE_AVG_AR_DELAY'] == 0 , 'DUMMY_DELAY'] = 0
count_calendar.loc[(count_calendar['DATE_AVG_AR_DELAY'] > 0) & (count_calendar['DATE_AVG_AR_DELAY'] <= 15), 'DUMMY_DELAY'] = 0.5
count_calendar.loc[(count_calendar['DATE_AVG_AR_DELAY'] > 15) & (count_calendar['DATE_AVG_AR_DELAY'] <= 30) , 'DUMMY_DELAY'] = 1
count_calendar.loc[(count_calendar['DATE_AVG_AR_DELAY'] > 30) & (count_calendar['DATE_AVG_AR_DELAY'] <= 45) , 'DUMMY_DELAY'] = 1.5
count_calendar.loc[count_calendar['DATE_AVG_AR_DELAY'] > 45 , 'DUMMY_DELAY'] = 2

In [ ]:
count_calendar['MONTH_DE'].replace({3: "March", 4: "April", 5: "May", 6: "June", 7: "July"}, inplace=True)

In [ ]:
count_calendar = count_calendar[::-1]
count_calendar.head()

In [ ]:

fig = go.Figure(data=go.Heatmap(
        z=count_calendar['DUMMY_DELAY'],
        y=count_calendar['MONTH_DE'],
        x=count_calendar['DAY_DE'],
        colorscale='balance'))

fig.update_layout(
    title='Average Delay by Date',
    xaxis_nticks=60,
    yaxis_nticks=8)

fig.show()

In [ ]:
calendar_delays.head()

In [ ]:
time_ar_delays = calendar_delays.groupby(["HOUR_DE"], as_index=False).mean()
time_ar_count = calendar_delays.groupby(["HOUR_DE"], as_index=False).size()

In [ ]:
time_ar_delays['FLIGHT_COUNT'] = time_ar_count['size']
time_ar_delays['ANGLE'] = time_ar_count["HOUR_DE"] * 15

In [ ]:
time_ar_delays

In [ ]:
fifteens = []
times = []
for i in range(len(time_ar_delays)):
    fifteens.append(i*15)
    times.append(i)

In [ ]:
fig = px.bar_polar(time_ar_delays, r='FLIGHT_COUNT', theta="ANGLE", color="ARRIVAL_DELAY_v1",
                   color_discrete_sequence= px.colors.sequential.RdBu,
                   title="Arrival Delays Based on Flight Departure Times"
                  )

fig.update_layout(
    polar = dict(
      radialaxis = dict(type = "log", tickangle = 45),
      angularaxis = dict(
            tickmode="array",
            tickvals=fifteens,
            ticktext=times
            )
    ))

fig.show()